# Analysis of mass training of models

### Combine all results into one dataframe

In [51]:
import os
import pandas as pd
import plotly.graph_objects as go
import numpy as np

In [52]:
training_results = "./TRAINING_RESULTS"

image_files_FINAL  = [file[12:-4] for file in os.listdir(f"{training_results}")]
image_files_FINAL

['DeepLabV3Plus__mobilenet_v2__DiceLoss__Freeze_encoder_False',
 'DeepLabV3Plus__mobilenet_v2__DiceLoss__Freeze_encoder_True',
 'DeepLabV3Plus__mobilenet_v2__JaccardLoss__Freeze_encoder_False',
 'DeepLabV3Plus__mobilenet_v2__JaccardLoss__Freeze_encoder_True',
 'DeepLabV3Plus__resnet101__DiceLoss__Freeze_encoder_False',
 'DeepLabV3Plus__resnet101__DiceLoss__Freeze_encoder_True',
 'DeepLabV3Plus__resnet101__JaccardLoss__Freeze_encoder_False',
 'DeepLabV3Plus__resnet101__JaccardLoss__Freeze_encoder_True',
 'FPN__mobilenet_v2__DiceLoss__Freeze_encoder_False',
 'FPN__mobilenet_v2__DiceLoss__Freeze_encoder_True',
 'FPN__mobilenet_v2__JaccardLoss__Freeze_encoder_False',
 'FPN__mobilenet_v2__JaccardLoss__Freeze_encoder_True',
 'FPN__resnet101__DiceLoss__Freeze_encoder_False',
 'FPN__resnet101__DiceLoss__Freeze_encoder_True',
 'FPN__resnet101__JaccardLoss__Freeze_encoder_False',
 'FPN__resnet101__JaccardLoss__Freeze_encoder_True',
 'Linknet__mobilenet_v2__DiceLoss__Freeze_encoder_False',
 'Link

# Combine into one dataframe

In [53]:
big_df: pd.DataFrame = None

for f in image_files_FINAL:
    architecture, encoder, loss, encoder_freeze = f.split('__')
    df = pd.read_csv(f"./TRAINING_RESULTS/best_model__{architecture}__{encoder}__{loss}__{encoder_freeze}.csv", index_col=0)
    df['architecture'] = architecture
    df['encoder'] = encoder
    df['loss_func'] = loss
    if type(big_df) == 'NoneType':
        big_df = df
    else:
        big_df = pd.concat((big_df, df))

big_df['memory_used'] /= (1024*1024)

In [54]:
l = big_df.groupby(['architecture', 'encoder', 'loss_func', 'freeze_encoder'])
for g in l:
    print(g[1].iloc[g[1]['val_iou_score'].idxmax()])
    break

train_iou_score         0.926577
train_accuracy          0.978418
train_fscore            0.961751
train_loss              0.057631
val_iou_score           0.925391
val_accuracy            0.978577
val_fscore              0.960956
val_loss                0.054327
parameters               4378513
memory_used               5354.0
freeze_encoder             False
architecture       DeepLabV3Plus
encoder             mobilenet_v2
loss_func               DiceLoss
Name: 5, dtype: object


In [55]:
big_df[:5]

,train_iou_score,train_accuracy,train_fscore,train_loss,val_iou_score,val_accuracy,val_fscore,val_loss,parameters,memory_used,freeze_encoder,architecture,encoder,loss_func
0,0.790320,0.924422,0.871059,0.275363,0.889814,0.966431,0.941087,0.130975,4378513,5354.0,False,DeepLabV3Plus,mobilenet_v2,DiceLoss
1,0.890091,0.966504,0.941300,0.116404,0.909866,0.973333,0.952406,0.092867,4378513,5354.0,False,DeepLabV3Plus,mobilenet_v2,DiceLoss
2,0.906770,0.972362,0.950674,0.090420,0.910591,0.973462,0.952898,0.076593,4378513,5354.0,False,DeepLabV3Plus,mobilenet_v2,DiceLoss
3,0.916283,0.975439,0.955943,0.075271,0.877825,0.961667,0.934357,0.087910,4378513,5354.0,False,DeepLabV3Plus,mobilenet_v2,DiceLoss
4,0.916649,0.975170,0.956265,0.066970,0.919754,0.976632,0.957922,0.062395,4378513,5354.0,False,DeepLabV3Plus,mobilenet_v2,DiceLoss


# Plot

In [56]:
def plot_data(big_df: pd.DataFrame, y_volumn, y_column_name, plot_title, x_column="index", x_column_name="Epochs", filter=None):
    fig = go.Figure()
    big_df_copy = big_df.copy(deep=True)
    if filter != None:
        big_df_copy.query(filter, inplace=True)
    # for f in image_files_FINAL:
    for data in big_df_copy.groupby(['architecture', 'encoder', 'loss_func', 'freeze_encoder']):
        architecture, encoder, loss, freeze = data[0]
        # fig.add_trace(go.Scatter(x=df.index+1, y=df['train_iou_score'], name=f"{architecture} {encoder} {loss}", hoverinfo='text+name',)) # 
        # fig.add_trace(go.Scatter(x=data[1].index+1, y=data[1]['val_iou_score'], name=f"{architecture} {encoder} {loss}", hoverinfo='y+text+name',)) # 
        hovertemplate=f"<b>{architecture} {encoder} {loss} {freeze}</b><br><br>" + \
                        f"{plot_title}"+": %{y:.3}<br>" + \
                        f"{x_column_name}"+": %{x}<br>" + \
                        "<extra></extra>",
        if x_column == "index":
            fig.add_trace(go.Scatter(x=data[1].index+1, y=data[1][y_volumn], name=f"{architecture} {encoder} {loss} {freeze}", hovertemplate=hovertemplate)) # 
        else:
            fig.add_trace(go.Scatter(x=data[1][x_column], y=data[1][y_volumn], name=f"{architecture} {encoder} {loss} {freeze}", hovertemplate=hovertemplate, mode='markers')) # 
        
    # fig.update_traces(hoverinfo='text+name', mode='lines+markers')
    fig.update_layout(legend=dict(y=0.5, traceorder='reversed', font_size=16))
    fig.update_layout(
        title=f"{plot_title} - (Architecture Encoder Loss)",
        # title="Training loss - (Architecture Encoder Loss)",
        xaxis_title=x_column_name,
        yaxis_title=y_column_name,
            font=dict(
            size=14,
        )
    )
    # fig.update_layout(hovermode="x unified")
    fig.show()

In [57]:
# Loss
plot_data(big_df, y_volumn='train_loss', y_column_name="Jaccard Loss", plot_title="Train loss", filter='loss_func == "JaccardLoss"')
plot_data(big_df, y_volumn='val_loss', y_column_name="Jaccard Loss", plot_title="Val loss", filter='loss_func == "JaccardLoss"')
# IoU
plot_data(big_df, y_volumn='train_iou_score', y_column_name="IoU", plot_title="Train IoU", filter='loss_func == "JaccardLoss"')
plot_data(big_df, y_volumn='val_iou_score',  y_column_name= "IoU", plot_title="Val IoU", filter='loss_func == "JaccardLoss"')

# Memory consumption

In [58]:
def plot_data_memory(big_df: pd.DataFrame, y_volumn, y_column_name, plot_title, x_column="index", x_column_name="Epochs", filter=None):
    fig = go.Figure()
    big_df_copy = big_df.copy(deep=True)
    if filter != None:
        big_df_copy.query(filter, inplace=True)
    # for f in image_files_FINAL:
    for data in big_df_copy.groupby(['architecture', 'encoder', 'loss_func', 'freeze_encoder']):
        architecture, encoder, loss, freeze = data[0]
        df: pd.DataFrame = data[1]
        max_row = df.iloc[df['val_iou_score'].idxmax()]

        hovertemplate=f"<b>{architecture} {encoder} {loss} {freeze}</b><br><br>" + \
                        f"{plot_title}"+": %{y:.3}<br>" + \
                        f"{x_column_name}"+": %{x}<br>" + \
                        "<extra></extra>",
        
        fig.add_trace(go.Scatter(x=[max_row[x_column]], y=[max_row[y_volumn]], name=f"{architecture} {encoder} {loss} {freeze}", hovertemplate=hovertemplate, mode='markers')) # 
        
    # fig.update_traces(hoverinfo='text+name', mode='lines+markers')
    fig.update_layout(legend=dict(y=0.5, traceorder='reversed', font_size=16))
    fig.update_layout(
        title=f"{plot_title} - (Architecture Encoder Loss EncoderFreeze)",
        # title="Training loss - (Architecture Encoder Loss)",
        xaxis_title=x_column_name,
        yaxis_title=y_column_name,
            font=dict(
            size=14,
        )
    )
    # fig.update_layout(hovermode="x unified")
    fig.show()

# plot_data_memory(big_df, y_volumn='train_iou_score', y_column_name="IoU", x_column="memory_used", x_column_name="Memory (mb)", plot_title="Train IoU")
plot_data_memory(big_df, y_volumn='val_iou_score', y_column_name="Val IoU", x_column="memory_used", x_column_name="Memory (mb)", plot_title="Val IoU vs GPU memory")